In [ ]:
# Colab에서 실행하기 위한 설치
!pip install langchain langchain-openai chromadb sentence-transformers langchain_community


In [ ]:

import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

# OpenAI API 키 설정 (본인의 API 키로 변경)
from dotenv import load_dotenv

# load_dotenv()
# OpenAI API 클라이언트 생성
# API_KEY = os.getenv("OPENAI_API_KEY")

from google.colab import userdata
api_key=userdata.get('api_key')
os.environ["OPENAI_API_KEY"] = api_key

# 한글 문서 데이터 준비
documents = [
    """
한국의 전통 음식 중 하나인 김치는 배추, 무, 오이 등의 채소를 소금에 절인 후
고춧가루, 마늘, 생강, 젓갈 등의 양념을 넣어 발효시킨 음식입니다.
김치는 유산균이 풍부하여 장 건강에 좋으며, 비타민 C와 식이섬유가 많이 들어있어
면역력 증진에 도움이 됩니다. 한국인의 식탁에서 빠질 수 없는 대표적인 반찬이죠.
""",
    """
불고기는 한국의 대표적인 고기 요리로, 얇게 썬 소고기를 간장, 설탕, 마늘, 배즙 등으로
양념하여 구운 음식입니다. 달콤하고 짭짤한 맛이 특징이며,
한국을 방문하는 외국인들이 가장 선호하는 한국 음식 중 하나입니다.
불고기는 조선시대부터 즐겨 먹었던 전통 음식으로 역사가 깊습니다.
""",
    """
비빔밥은 밥 위에 여러 가지 나물과 고기, 계란 등을 올리고 고추장을 넣어 비벼 먹는
한국의 대표 음식입니다. 전주 비빔밥이 특히 유명하며,
영양 균형이 잘 맞춰진 건강한 음식으로 알려져 있습니다.
다양한 채소와 나물이 들어가 비타민과 미네랄이 풍부합니다.
""",
    """
한국의 전통 발효 음식인 된장은 콩을 삶아서 메주를 만든 후 소금물에 담가 발효시켜 만듭니다.
된장찌개, 된장국 등 다양한 요리에 사용되며, 단백질과 필수 아미노산이 풍부합니다.
또한 이소플라본이 들어있어 심혈관 질환 예방에 도움이 된다고 알려져 있습니다.
""",
    """
한복은 한국의 전통 의상으로, 여성용 한복은 저고리와 치마로,
남성용 한복은 저고리와 바지로 구성됩니다.
자연스럽고 우아한 곡선미가 특징이며, 명절이나 특별한 날에 착용합니다.
최근에는 현대적으로 개량된 생활한복도 인기를 끌고 있습니다.
"""
]

# Document 객체로 변환
docs = [Document(page_content=doc) for doc in documents]

# 텍스트 분할기 설정
text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
split_docs = text_splitter.split_documents(docs)

# 임베딩 및 벡터 스토어 생성
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

# LLM 초기화
llm = OpenAI(temperature=0)

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

# 테스트 질문들
questions = [
    "김치의 건강상 효능은 무엇인가요?",
    "불고기는 어떤 양념을 사용하나요?",
    "비빔밥에는 어떤 재료들이 들어가나요?",
    "한복의 구성 요소는 무엇인가요?",
    "된장의 주요 영양소는 무엇인가요?"
]

print("=== 한글 문서 QA 시스템 테스트 ===\n")

for i, question in enumerate(questions, 1):
    print(f"질문 {i}: {question}")

    result = qa_chain({"query": question})

    print(f"답변: {result['result']}")
    print(f"참조 문서 수: {len(result['source_documents'])}")
    print("-" * 50)

# 벡터 스토어 저장 (재사용 가능)
vectorstore.persist()
print("벡터 데이터베이스가 './chroma_db'에 저장되었습니다.")

/var/folders/xn/w4kqb1715fz7h49tp_h7hbmr0000gn/T/ipykernel_65408/2884784572.py:60: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.